# Opimisation thèse Manon

### Import des librairies

In [1]:
import numpy as np
import matplotlib.pyplot as plt

## Présentation du problème

On souhaite résoudre le problème suivant :
$$
    \min_{x\in K} f(x), \quad \text{with } K=\{x\in(\mathbb{R}_+^*)^n \vert h(x)\geq 0\}.
$$
Avec :
$$
    f(x)=\sum_{i=1}^n x_i, \quad h(x)=\text{Module de stabilité de la matrice }A(x).
$$

## Définitions basiques

### Elements utiles

In [3]:
# Matrice D (points du graph)
D12=1.4
D21=0.8
D13 = 0.8
D31 = 0.2
D32 = 0.6
D23 = 0.5

D = np.array([[-D21-D31,D12,D13],
              [D21,-D12-D32,D23],
              [D31,D32,-D13-D23]])
# D42 = 0; D52 = 0; D24 = 0; D25=0;D43=0;D53=0;D34=0;D35=0

# Constantes et fonctions
n = np.shape(D)[0] 
I = np.eye(n)
geneGFonc = lambda a,K: lambda x: a-x/K

### Matrice A(x)

In [4]:
g1= geneGFonc(0.7, 2)
g2= geneGFonc(0.5, 3)
g3= geneGFonc(0.2, 4)

def A(x):
    x1 = x[0,0]
    x2 = x[0,1]
    return np.diag([g1(0)-x1,g2(0)-x2,g3(0)]) +D

### Module de stabilité (contrainte)

In [ ]:
def h(x):
    eigv, eigp = np.linalg.eig(A(x))
    real = eigv.real
    return max(real)

### Coût et affichage